In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import openpyxl
from time import sleep

numero_oab = "Adicione o numero"

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get('https://pje-consulta-publica.tjmg.jus.br/')
sleep(5)
# digitar numero AOB
campo_oab = driver.find_element(
    By.XPATH, '//*[@id="fPP:Decoration:numeroOAB"]')
campo_oab.send_keys(numero_oab)

# selecionar estado
dropdown_estados = driver.find_element(
    By.XPATH, '//*[@id="fPP:Decoration:estadoComboOAB"]')
opcoes_estados = Select(dropdown_estados)
opcoes_estados.select_by_visible_text("SP")

# clicar em pesquisar
botao_pesquisar = driver.find_element(
    By.XPATH, "//*[@id='fPP:searchProcessos']")
botao_pesquisar.click()
sleep(5)

# Clicar em cada um dos processos
processos = driver.find_elements(By.XPATH, "//b[@class='btn-block']")
for processo in processos:
    processo.click()
    sleep(5)
    # interage entre janelas
    janelas = driver.window_handles
    driver.switch_to.window(janelas[-1])
    driver.set_window_size(1920, 1080)

    # coleta somente o texto da numero do processo
    numero_processo = driver.find_elements(
        By.XPATH, "//div[@class='col-sm-12 ']")
    numero_processo = numero_processo[0]
    numero_processo = numero_processo.text
    # coleta somente o texto da data de distribicao
    data_distribuicao = driver.find_elements(
        By.XPATH, "//div[@class='value col-sm-12 ']")
    data_distribuicao = data_distribuicao[1]
    data_distribuicao = data_distribuicao.text

    # Extrair e Guardar todas as ultimas movimentaçoes
    movimentacoes = driver.find_elements(
        By.XPATH, "//div[@id='j_id132:processoEventoPanel_body']//tr[contains(@class,'rich-table-row')]//td//div//div//span")
    lista_movimentacoes = []
    for movimentacao in movimentacoes:
        lista_movimentacoes.append(movimentacao.text)

    # guardar na planilha
    workbook = openpyxl.load_workbook('dados.xlsx')
    try:
        # Codigo para inserir dados em pagina existente e inserir as informações
        # acessar pagina do processo
        pagina_processo = workbook[numero_processo]
        # criar nome das colunas
        pagina_processo['A1'].value = "Numero Processo"
        pagina_processo['B1'].value = "Data Distribuicao"
        pagina_processo['C1'].value = "Movimentacoes"
        # adicionar numero do processo
        pagina_processo['A2'].value = numero_processo
        # adicionar data de distribuicao
        pagina_processo['B2'].value = data_distribuicao
        # adicionar movimentação
        for index, linha in enumerate(pagina_processo.iter_rows(min_row=2, max_row=len(lista_movimentacoes), min_col=3, max_col=3)):
            for celula in linha:
                celula.value = lista_movimentacoes[index]
        workbook.save('dados.xlsx')
        driver.close()
        sleep(5)
        driver.switch_to.window(driver.window_handles[0])
        print("Finalizado")
    except Exception as error:
        # código para criar uma pagina do zero e inserir as informações
        workbook.create_sheet(numero_processo)
        # Codigo para inserir dados em pagina existente e inserir as informações
        # acessar pagina do processo
        pagina_processo = workbook[numero_processo]
        # criar nome das colunas
        pagina_processo['A1'].value = "Numero Processo"
        pagina_processo['B1'].value = "Data Distribuicao"
        pagina_processo['C1'].value = "Movimentacoes"
        # adicionar numero do processo
        pagina_processo['A2'].value = numero_processo
        # adicionar data de distribuicao
        pagina_processo['B2'].value = data_distribuicao
        # adicionar movimentação
        for index, linha in enumerate(pagina_processo.iter_rows(min_row=2, max_row=len(lista_movimentacoes), min_col=3, max_col=3)):
            for celula in linha:
                celula.value = lista_movimentacoes[index]
        workbook.save('dados.xlsx')
        driver.close()
        sleep(5)
        driver.switch_to.window(driver.window_handles[0])
        print("Finalizado")


ModuleNotFoundError: No module named 'selenium'